# How to build a tool-using agent with LangChain

This notebook takes you through how to use LangChain to augment an OpenAI model with access to external tools. In particular, you'll be able to create LLM agents that use custom tools to answer user queries.


## What is Langchain?
[LangChain](https://python.langchain.com/en/latest/index.html) is a framework for developing applications powered by language models. Their framework enables you to build layered LLM-powered applications that are context-aware and able to interact dynamically with their environment as agents, leading to simplified code for you and a more dynamic user experience for your customers.

## Why do LLMs need to use Tools?
One of the most common challenges with LLMs is overcoming the lack of recency and specificity in their training data - answers can be out of date, and they are prone to hallucinations given the huge variety in their knowledge base. Tools are a great method of allowing an LLM to answer within a controlled context that draws on your existing knowledge bases and internal APIs - instead of trying to prompt engineer the LLM all the way to your intended answer, you allow it access to tools that it calls on dynamically for info, parses, and serves to customer.

Providing LLMs access to tools can enable them to answer questions with context directly from search engines, APIs or your own databases. Instead of answering directly, an LLM with access to tools can perform intermediate steps to gather relevant information. Tools can also be used in combination. [For example](https://python.langchain.com/en/latest/modules/agents/agents/examples/mrkl_chat.html), a language model can be made to use a search tool to lookup quantitative information and a calculator to execute calculations.

## Notebook Sections

- **Setup:** Import packages and connect to a Pinecone vector database.
- **LLM Agent:** Build an agent that leverages a modified version of the [ReAct](https://react-lm.github.io/) framework to do chain-of-thought reasoning.
- **LLM Agent with History:** Provide the LLM with access to previous steps in the conversation.
- **Knowledge Base:** Create a knowledge base of "Stuff You Should Know" podcast episodes, to be accessed through a tool.
- **LLM Agent with Tools:** Extend the agent with access to multiple tools and test that it uses them to answer questions.

In [ ]:
%load_ext autoreload
%autoreload 2

# Setup

Import libraries and set up a connection to a [Pinecone](https://www.pinecone.io) vector database.

You can substitute Pinecone for any other vectorstore or database - there are a [selection](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html) that are supported by Langchain natively, while other connectors will need to be developed yourself.

In [ ]:
!pip install openai
!pip install pinecone-client
!pip install pandas
!pip install typing
!pip install tqdm
!pip install langchain
!pip install wget
!pip install google-search-results
!pip install tiktoken

  Using cached langchain-0.1.16-py3-none-any.whl (817 kB)
  Using cached dataclasses_json-0.6.4-py3-none-any.whl (28 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
  Using cached langchain_community-0.0.33-py3-none-any.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.1/289.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=2d886ae29cad240c4f1fdf672402d8bbaebe9f6fb290bbec3c81a6914614f478

In [ ]:
import datetime
import json
import openai
import os
import pandas as pd
from pinecone import Pinecone
import re
from tqdm.auto import tqdm
from typing import List, Union
import zipfile

# Langchain imports
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate, ChatPromptTemplate
from langchain import SerpAPIWrapper, LLMChain
from langchain.schema import AgentAction, AgentFinish, HumanMessage, SystemMessage
# LLM wrapper
from langchain.chat_models import ChatOpenAI
from langchain import OpenAI
# Conversational memory
from langchain.memory import ConversationBufferWindowMemory
# Embeddings and vectorstore
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

# Vectorstore Index
index_name = 'podcast'

In [ ]:
# @title
import os
os.environ['OPENAI_API_KEY'] = 'sk-' # Put your openai key here
os.environ['PINECONE_API_KEY'] = '' # Put your pineconde key here
os.environ['SERPAPI_API_KEY'] = '' # Put your serpapi key here
os.environ['PINECONE_ENVIRONMENT'] = 'asia-southeast1-gcp-free' # Put the pinecone environment


For acquiring an API key to connect with Pinecone, you can set up a [free account](https://app.pinecone.io/) and store it in the `api_key` variable below or in your environment variables under `PINECONE_API_KEY`

In [ ]:
api_key = os.getenv("PINECONE_API_KEY") or "PINECONE_API_KEY"

# find environment next to your API key in the Pinecone console
env = os.getenv("PINECONE_ENVIRONMENT") or "PINECONE_ENVIRONMENT"
import os
from pinecone import Pinecone, ServerlessSpec


pinecone = Pinecone(
    api_key=os.environ.get("PINECONE_API_KEY")
)

pinecone.create_index(
  name=index_name,
  dimension=1536,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  )
)

index = pinecone.Index(name=index_name)

In [ ]:
pinecone.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'podcast-b65090a.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'podcast',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

## LLM Agent

An [LLM agent](https://python.langchain.com/en/latest/modules/agents/agents/custom_llm_agent.html) in Langchain has many configurable components, which are detailed in the Langchain documentation.

We'll employ a few of the core concepts to make an agent that talks in the way we want, can use tools to answer questions, and uses the appropriate language model to power the conversation.
- **Prompt Template:** The input template to control the LLM's behaviour and how it accepts inputs and produces outputs - this is the brain that drives your application ([docs](https://python.langchain.com/en/latest/modules/prompts/prompt_templates.html)).
- **Output Parser:** A method of parsing the output from the prompt. If the LLM produces output using certain headers, you can enable complex interactions where variables are generated by the LLM in their response and passed into the next step of the chain ([docs](https://python.langchain.com/en/latest/modules/prompts/output_parsers.html)).
- **LLM Chain:** A Chain brings together a prompt template with an LLM that will execute it - in this case we'll be using ```gpt-3.5-turbo``` but this framework can be used with OpenAI completions models, or other LLMs entirely ([docs](https://python.langchain.com/en/latest/modules/chains.html)).
- **Tool:** An external service that the LLM can use to retrieve information or execute commands should the user require it ([docs](https://python.langchain.com/en/latest/modules/agents/tools.html)).
- **Agent:** The glue that brings all of this together, an agent can call multiple LLM Chains, each with their own tools. Agents can be extended with your own logic to allow retries, error handling and any other methods you choose to add reliability to your application ([docs](https://python.langchain.com/en/latest/modules/agents.html)).

**NB:** Before using this cookbook with the Search tool you'll need to sign up on https://serpapi.com/ and generate an API key. Once you have it, store it in an environment variable named ```SERPAPI_API_KEY```

In [ ]:
# Initiate a Search tool - note you'll need to have set SERPAPI_API_KEY as an environment variable as per the above instructions
search = SerpAPIWrapper()

# Define a list of tools
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [ ]:
# Set up the prompt with input variables for tools, user input and a scratchpad for the model to record its workings
template = """Answer the following questions as best you can, but speaking as a financial analyst might speak. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a financial analyst when giving your final answer. Use lots of numbers and metrics

Question: {input}
{agent_scratchpad}"""

In [ ]:
# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)

        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "

        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts

        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])

        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]

prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

In [ ]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:

        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )

        # Parse out the action and action input
        regex = r"Action: (.*?)[\n]*Action Input:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)

        # If it can't parse the output it raises an error
        # You can add your own logic here to handle errors in a different way i.e. pass to a human, give a canned response
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)

        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser = CustomOutputParser()

In [ ]:
# Initiate our LLM - default is 'gpt-3.5-turbo'
llm = ChatOpenAI(temperature=0)

# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Using tools, the LLM chain and output_parser to make an agent
tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    # We use "Observation" as our stop sequence so it will stop when it receives Tool output
    # If you change your prompt template you'll need to adjust this as well
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [ ]:
# Initiate the agent that will respond to our queries
# Set verbose=True to share the CoT reasoning the LLM goes through
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.run("How has Apple's stock performed compared to the NASDAQ index in 2023?")



> Entering new AgentExecutor chain...
Thought: I need to compare the performance of Apple's stock to the NASDAQ index in 2023 to provide an accurate analysis.
Action: Search
Action Input: "Apple stock performance vs NASDAQ index 2023"

Observation:['Apple stock is down 5.3% so far in 2024 compared to a 5.1% gain in the Nasdaq Composite. Over the last year, Apple is up 18.6% compared to the ...', 'Technology stocks got off to a decent start in 2024 -- the Nasdaq-100 index has clocked a 5% gain year to date as of this writing. But Apple ...', 'Apple (NASDAQ:AAPL) stock has witnessed a rocky start to 2024, declining by about 4% year-to-date, falling to about $185 per share.', 'Since March 1, 2021, Apple (NASDAQ: AAPL) shares have climbed almost 50%. That gain is more than double the increase in the Nasdaq Composite ...', 'Apple (NASDAQ:AAPL) stock has fared well this year, rising by about 45% year-to-date despite a tough demand environment following the easing ...', "The Nasdaq composit

"Apple's stock has underperformed compared to the NASDAQ index in 2023."

In [ ]:
agent_executor.run("How about Apple's stock performed compared to the NASDAQ index in 2022?")



> Entering new AgentExecutor chain...
Thought: I need to compare the performance of Apple's stock to the NASDAQ index in 2022 to provide an analysis.
Action: Search
Action Input: Apple stock performance vs NASDAQ index 2022

Observation:['The shares of Apple (NASDAQ:AAPL) had a tough 2022, declining by 27% over the year, underperforming the S&P 500 which was down by about 20% ...', 'Apple stock is down 5.3% so far in 2024 compared to a 5.1% gain in the Nasdaq Composite. Over the last year, Apple is up 18.6% compared to the ...', 'Discover real-time Apple Inc. Common Stock (AAPL) stock prices, quotes, historical data, news, and Insights for informed trading and investment decisions.', 'Apple (NASDAQ:AAPL) stock has witnessed a rocky start to 2024, declining by about 4% year-to-date, falling to about $185 per share.', 'NASDAQ: AAPL ; March 13, 2024, $171.13 ; March 12, 2024, $173.23 ; March 11, 2024, $172.75 ; March 08, 2024, $170.73.', 'Through Friday, Apple was underperforming the Na

"Apple's stock performed worse compared to the NASDAQ index in 2022, with a decline of 27% for Apple versus a 20% decline for the NASDAQ index."

In [ ]:
agent_executor.run("How about Apple's stock performed compared to the NASDAQ index in the years 2019,2020,2021?")



> Entering new AgentExecutor chain...
Thought: I need to compare Apple's stock performance to the NASDAQ index over the past three years to provide an analysis.
Action: Search
Action Input: Apple stock performance compared to NASDAQ index 2019

Observation:["Shares of Apple (NASDAQ: AAPL) gained 86.2% in 2019, according to data from S&P Global Market Intelligence . The tech stock's share price ...", 'Find the latest historical data for Apple Inc. Common Stock (AAPL) at Nasdaq.com. View historical data in a monthly, bi-annual, or yearly format.', "Stock Quote: NASDAQ: AAPL ; Day's Open175.36 ; Closing Price172.69 ; Volume73.1 ; Intraday High176.63 ; Intraday Low172.50.", 'The Nasdaq index has an average forward price-to-earnings ratio of roughly 21, while Apple has a forward price-to-earnings ratio of roughly 36.', "Apple and Microsoft together accounted for nearly 15% of the S&P 500's advance in 2019, according to S&P Dow Jones Indices.", "Apple Inc (AAPL) Stock Against Nasdaq Compos

"Based on the data collected, Apple's stock outperformed the NASDAQ index in 2019 with a gain of 86.2%. In 2020, Apple's stock was down 5.3% compared to a 5.1% gain in the NASDAQ Composite. In 2021, Apple's stock performance was positive, but the exact comparison to the NASDAQ index is not provided. Overall, Apple's stock has shown strong performance over the past three years, with fluctuations in comparison to the NASDAQ index."

## LLM Agent with History

Extend the LLM Agent with the ability to retain a [memory](https://python.langchain.com/en/latest/modules/agents/agents/custom_llm_agent.html#adding-memory) and use it as context as it continues the conversation.

We use a simple ```ConversationBufferWindowMemory``` for this example that keeps a rolling window of the last two conversation turns. LangChain has other [memory options](https://python.langchain.com/en/latest/modules/memory.html), with different tradeoffs suitable for different use cases.

In [ ]:
# Set up a prompt template which can interpolate the history
template_with_history = """You are SearchGPT, a professional search engine who provides informative answers to users. Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to give detailed, informative answers

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

In [ ]:
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # The history template includes "history" as an input variable so we can interpolate it into the prompt
    input_variables=["input", "intermediate_steps", "history"]
)

llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [ ]:
# Initiate the memory with k=2 to keep the last two turns
# Provide the memory to the agent
memory = ConversationBufferWindowMemory(k=2)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

In [ ]:
agent_executor.run("How did the US stock market perform in 2022?")



> Entering new AgentExecutor chain...
Question: How did the US stock market perform in 2022?
Thought: I should search for the latest information on the performance of the US stock market in 2022.
Action: Search
Action Input: "US stock market performance 2022"

Observation:The DJIA fell 18.78% since its January 4 high, while the Nasdaq Composite fell 33.70% from its November 19 high. On September 13, 2022, the S&P 500 experienced its largest single-day drop since June 2020.
The US stock market experienced significant declines in 2022, with major indices like the DJIA, Nasdaq Composite, and S&P 500 all seeing drops throughout the year. These declines were influenced by various factors such as inflation concerns, geopolitical tensions, and uncertainty surrounding interest rates. Overall, it was a challenging year for investors in the US stock market.

Final Answer: The US stock market performed poorly in 2022, with major indices like the DJIA, Nasdaq Composite, and S&P 500 experiencing 

'The US stock market performed poorly in 2022, with major indices like the DJIA, Nasdaq Composite, and S&P 500 experiencing significant declines.'

In [ ]:
agent_executor.run("how about in 2021?")



> Entering new AgentExecutor chain...
Question: How did the US stock market perform in 2021?
Thought: I should search for information on the performance of the US stock market in 2021.
Action: Search
Action Input: "US stock market performance in 2021"

Observation:Stock market return (%, year-on-year) in United States was reported at 32.65 % in 2021, according to the World Bank collection of development indicators, compiled from officially recognized sources.
The US stock market performed well in 2021, with a return of 32.65% for the year.
Final Answer: The US stock market had a return of 32.65% in 2021.

> Finished chain.


'The US stock market had a return of 32.65% in 2021.'

## Knowledge base

Create a custom vectorstore for the Agent to use as a tool to answer questions with. We'll store the results in [Pinecone](https://docs.pinecone.io/docs/quickstart), which is supported by LangChain ([Docs](https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/pinecone.html), [API reference](https://python.langchain.com/en/latest/reference/modules/vectorstore.html)). For help getting started with Pinecone or other vector databases, we have a [cookbook](https://github.com/openai/openai-cookbook/blob/colin/examples/vector_databases/Using_vector_databases_for_embeddings_search.ipynb) to help you get started.

You can check the LangChain documentation to see what other [vectorstores](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html) and [databases](https://python.langchain.com/en/latest/modules/chains/examples/sqlite.html) are available.

For this example we'll use the transcripts of the Stuff You Should Know podcast, which was provided thanks to OSF DOI [10.17605/OSF.IO/VM9NT](https://doi.org/10.17605/OSF.IO/VM9NT)

In [ ]:
import wget

# Here is a URL to a zip archive containing the transcribed podcasts
# Note that this data has already been split into chunks and embeddings from OpenAI's text-embedding-ada-002 embedding model are included
content_url = 'https://cdn.openai.com/API/examples/data/sysk_podcast_transcripts_embedded.json.zip'

# Download the file (it is ~541 MB so this will take some time)
wget.download(content_url)

'sysk_podcast_transcripts_embedded.json.zip'

In [ ]:
# Load podcasts
with zipfile.ZipFile("sysk_podcast_transcripts_embedded.json.zip","r") as zip_ref:
    zip_ref.extractall("./data")
f = open('./data/sysk_podcast_transcripts_embedded.json')
processed_podcasts = json.load(f)

In [ ]:
# Have a look at the contents
pd.DataFrame(processed_podcasts).head()

,id,filename,title,url,text_chunk,embedding,cleaned_id
0,sysk_with_transcripts_SYSK Selects How Crime S...,sysk_with_transcripts_SYSK Selects How Crime S...,\n\nSYSK Selects How Crime Scene Cleanup Works,https://chtbl.com/track/5899E/podtrac.com/pts/...,Title: sysk_with_transcripts_SYSK Selects How ...,"[0.021279960870742798, -0.005817972123622894, ...",sysk_with_transcripts_SYSK Selects How Crime S...
1,sysk_with_transcripts_SYSK Selects How Crime S...,sysk_with_transcripts_SYSK Selects How Crime S...,\n\nSYSK Selects How Crime Scene Cleanup Works,https://chtbl.com/track/5899E/podtrac.com/pts/...,Title: sysk_with_transcripts_SYSK Selects How ...,"[0.013859338127076626, 0.00857278611510992, 0....",sysk_with_transcripts_SYSK Selects How Crime S...
2,sysk_with_transcripts_SYSK Selects How Crime S...,sysk_with_transcripts_SYSK Selects How Crime S...,\n\nSYSK Selects How Crime Scene Cleanup Works,https://chtbl.com/track/5899E/podtrac.com/pts/...,Title: sysk_with_transcripts_SYSK Selects How ...,"[0.015242221765220165, 0.016030369326472282, 0...",sysk_with_transcripts_SYSK Selects How Crime S...
3,sysk_with_transcripts_SYSK Selects How Crime S...,sysk_with_transcripts_SYSK Selects How Crime S...,\n\nSYSK Selects How Crime Scene Cleanup Works,https://chtbl.com/track/5899E/podtrac.com/pts/...,Title: sysk_with_transcripts_SYSK Selects How ...,"[0.004371842369437218, -0.003036574460566044, ...",sysk_with_transcripts_SYSK Selects How Crime S...
4,sysk_with_transcripts_SYSK Selects How Crime S...,sysk_with_transcripts_SYSK Selects How Crime S...,\n\nSYSK Selects How Crime Scene Cleanup Works,https://chtbl.com/track/5899E/podtrac.com/pts/...,Title: sysk_with_transcripts_SYSK Selects How ...,"[0.017309172078967094, 0.015154214575886726, 0...",sysk_with_transcripts_SYSK Selects How Crime S...


In [ ]:
# Add the text embeddings to Pinecone

batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(processed_podcasts), batch_size)):
    # find end of batch
    i_end = min(len(processed_podcasts), i+batch_size)
    meta_batch = processed_podcasts[i:i_end]
    # get ids
    ids_batch = [x['cleaned_id'] for x in meta_batch]
    # get texts to encode
    texts = [x['text_chunk'] for x in meta_batch]
    # add embeddings
    embeds = [x['embedding'] for x in meta_batch]
    # cleanup metadata
    meta_batch = [{
        'filename': x['filename'],
        'title': x['title'],
        'text_chunk': x['text_chunk'],
        'url': x['url']
    } for x in meta_batch]
    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

  0%|          | 0/382 [00:00<?, ?it/s]

In [ ]:
# Configuring the embeddings to be used by our retriever to be OpenAI Embeddings, matching our embedded corpus
embeddings = OpenAIEmbeddings()

from langchain.vectorstores import Pinecone

# Loads a docsearch object from an existing Pinecone index so we can retrieve from it
docsearch = Pinecone.from_existing_index(index_name, embeddings, text_key='text_chunk')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
retriever = docsearch.as_retriever()

In [ ]:
query_docs = retriever.get_relevant_documents("Do I need a bank account?")

In [ ]:
# Print out the title and content for the most relevant retrieved documents
print("\n".join(['Title: ' + x.metadata['title'].strip() + '\n\n' + x.page_content + '\n\n' for x in query_docs]))

Title: sysk: Can You Live Without a Bank Account?

Title: sysk_with_transcripts_Can you live without a bank account.json;  And if you had a life, you didn't necessarily rectify your bank checkbook every day. Oh, wait, what is balancing a checkbook mean? Seriously? Yeah. Thank God for my wife. So another reason you might avoid a bank is philosophically. There may be a longstanding distrust of banks in your family that you don't want to put your money in, or you may just want to be like, you know what? I don't want to take part in this modern society. I want to kind of drop out a bit. And a really good first move is to shut your bank account down. That's a big statement. Oh, yeah, it is. But a lot of people that are underbanked and don't have accounts aren't there on purpose. It's not some philosophical statement. A lot of times it's simply because they are poor and they don't have a lot of alternatives. Yeah. And the other thing about not having a bank account, not only do you not have 

## LLM Agent with Tools

Extend our list of tools by creating a [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html) chain leveraging our Pinecone knowledge base.

In [ ]:
from langchain.chains import RetrievalQA

retrieval_llm = OpenAI(temperature=0)

podcast_retriever = RetrievalQA.from_chain_type(llm=retrieval_llm, chain_type="stuff", retriever=docsearch.as_retriever())

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [ ]:
expanded_tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name = 'Knowledge Base',
        func=podcast_retriever.run,
        description="Useful for general questions about how to do things and for details on interesting topics. Input should be a fully formed question."
    )
]

In [ ]:
# Re-initialize the agent with our new list of tools
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=expanded_tools,
    input_variables=["input", "intermediate_steps", "history"]
)
llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)
multi_tool_names = [tool.name for tool in expanded_tools]
multi_tool_agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=multi_tool_names
)

In [ ]:
multi_tool_memory = ConversationBufferWindowMemory(k=2)
multi_tool_executor = AgentExecutor.from_agent_and_tools(agent=multi_tool_agent, tools=expanded_tools, verbose=True, memory=multi_tool_memory)

In [ ]:
multi_tool_executor.run("Can you live without a bank account?")



> Entering new AgentExecutor chain...
Question: Can you live without a bank account?
Thought: This question involves understanding the necessity of a bank account in daily life.
Action: Knowledge Base
Action Input: "Can you live without a bank account?"

Observation: Yes, it is possible to live without a bank account, as approximately 7% of Americans do not have one. However, it may be more difficult to manage finances and access certain services without a bank account.
Living without a bank account may require alternative methods for managing finances and accessing services. Some people may choose to use prepaid debit cards, money orders, or check-cashing services instead. It's important to consider the potential limitations and challenges of not having a bank account. 

Final Answer: Yes, it is possible to live without a bank account, but it may present challenges in managing finances and accessing certain services.

> Finished chain.


'Yes, it is possible to live without a bank account, but it may present challenges in managing finances and accessing certain services.'

In [ ]:
multi_tool_executor.run("I would like to know if a bank account is needed in America")



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to consider the various reasons why someone might need a bank account in America.
Action: Search
Action Input: "reasons why a bank account is needed in America"

Observation:Opening a bank account can be one of the most important steps you take toward reaching your financial goals. Why? Because putting your money in an FDIC-insured bank account can offer you financial safety, easy access to your funds, savings from check-cashing fees, and overall financial peace of mind.
Based on the search results, it seems that having a bank account in America is important for financial safety, easy access to funds, savings from check-cashing fees, and overall financial peace of mind.
Final Answer: While it is possible to live without a bank account in America, having one can provide significant benefits in terms of financial security and convenience.

> Finished chain.


'While it is possible to live without a bank account in America, having one can provide significant benefits in terms of financial security and convenience.'

In [ ]:
multi_tool_executor.run('Can you tell me how much should you save for retirement?')



> Entering new AgentExecutor chain...
Question: Can you tell me how much should you save for retirement?
Thought: This question requires a general understanding of retirement savings guidelines.
Action: Knowledge Base
Action Input: "Retirement savings guidelines"

Observation: The general rule of thumb is to have three to six months worth of expenses saved in a separate emergency fund, followed by paying off any debt and then focusing on long-term savings such as a 401K or IRA. It is recommended to save at least eight times your ending salary by the time you retire. It is also important to start saving early and to diversify investments.
I now know the final answer
Final Answer: It is recommended to save at least eight times your ending salary by the time you retire.

> Finished chain.


'It is recommended to save at least eight times your ending salary by the time you retire.'

You now have a template to deploy conversational agents with tools. If you want to extend this with a Custom Agent to add your own retry behaviour or treatment of input/output variables, then follow [this article](https://python.langchain.com/en/latest/modules/agents/agents/custom_agent.html).

We look forward to seeing what you build!